In [ ]:
!pip install kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Importing dog vs cat dataset from kaggle

Extracting compressed file

In [ ]:
!kaggle competitions download -c dogs-vs-cats

dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
from zipfile import ZipFile

In [ ]:
dataset = "/content/dogs-vs-cats.zip"
with ZipFile(dataset,'r') as zip:
  zip.extractall()


In [ ]:
dataset = "/content/train.zip"
with ZipFile(dataset,'r') as zip:
  zip.extractall()

In [ ]:
dataset = "/content/test1.zip"
with ZipFile(dataset,'r') as zip:
  zip.extractall()

counting the no. of samples

In [ ]:
import os
path ,dirs ,files = next(os.walk('/content/train'))
file_count = len(files)
file_count

25000

Importing Dependencies

In [ ]:
import numpy as np
import PIL
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow
from PIL import Image

Displaying cat and dog images

In [ ]:
cat_img = mpimg.imread('/content/train/cat.10001.jpg')
dog_img = mpimg.imread('/content/train/dog.8298.jpg')

In [ ]:
cv2_imshow(cat_img)

In [ ]:
cv2_imshow(dog_img)

Since all the images of different size, we need to resize the images

In [ ]:
#counting the number of dog and cat images
filenames = os.listdir('/content/train')
for i in range(5):
  print(filenames[i])

dog.12457.jpg
cat.4047.jpg
cat.1596.jpg
cat.6179.jpg
dog.11234.jpg


In [ ]:
dog_c = 0
cat_c = 0
for img in filenames:
  if img[0:3] == 'cat':
    cat_c += 1
  if img[0:3]  == 'dog':
    dog_c +=1
print(dog_c)
print(cat_c)

In [ ]:
#resizing an image
os.mkdir('/content/resized') # making a folder for resized images



In [ ]:
original_fold = "/content/train/"
resized_fold = "/content/resized/"
file = os.listdir('/content/train')
for i in range (2000): # since we are using previoulsly trained mobilenet model, we do not need much data , so we are taking only 2000 images
    img_name = file[i]
    img_path = original_fold + img_name
    img = Image.open(img_path)
    img_resized = img.resize((224,224))
    img_resized = img_resized.convert('RGB')

    new_path = resized_fold + img_name

    img_resized.save(new_path)

In [ ]:
#checking no.of resized images
fname = os.listdir('/content/resized')
dc = 0
cc = 0
for i in fname:
    if i[0:3] == 'cat':
        cc +=1
    if i[0:3] == 'dog':
        dc +=1
print(cc)
print(dc)

1036
964


In [ ]:
dog = mpimg.imread("/content/resized/cat.1017.jpg")
plt.figure(figsize = (5,5))
plt.imshow(dog)
plt.show()

In [ ]:
cv2_imshow(dog)

Creating Labels
dog : 1
cat : 0

In [ ]:
fil_name = os.listdir('/content/resized')
y_train = []
for i in fil_name:
  if i[0:3] == 'cat':
    y_train.append(0)
  if i[0:3] == 'dog':
    y_train.append(1)


In [ ]:
for i in range(5):
  print(y_train[i])
  print(fil_name[i])

1
dog.12457.jpg
0
cat.4047.jpg
0
cat.1596.jpg
0
cat.6179.jpg
1
dog.11234.jpg


In [ ]:
dc1 = 0 #rechecking
for i in y_train:  # counting thwe no. of dog in Y_train as we know there are 981 dog imgs in training set
  if i == 1:
    dc1 +=1
print(dc1)

964


creating X_train

In [ ]:
fil = os.listdir('/content/resized')  # creating a list of file names

In [ ]:
x_train = []
for i in fil:
  img_path = '/content/resized/' + i
  img = mpimg.imread(img_path)
  x_train.append(img)


In [ ]:
X_train = np.array(x_train)
X_train.shape

(2000, 224, 224, 3)

In [ ]:
y_train = np.array(y_train)
type(y_train)

numpy.ndarray

In [ ]:
print(X_train[1])

In [ ]:
y_train.shape

(2000,)

In [ ]:
y_train[1]

0

In [ ]:
y_train = y_train.reshape(2000,1)

Train _Test _ Split

x_t = training
y_test = testing data

In [ ]:
x_t , x_test , y_t , y_test = train_test_split(X_train , y_train , test_size = 0.2 , random_state = 9)

In [ ]:
y_t.shape

(1600, 1)

In [ ]:
plt.figure(figsize = (10,10))
plt.imshow(x_t[5])
plt.show()

In [ ]:
y_t[5]

array([1])

Scaling the data

In [ ]:
x_t = x_t / 255
x_test = x_test / 255

In [ ]:
print(x_t[2])
print(x_test[2])

In [ ]:
print(y_t[1])

[1]


Building Model

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub  # for importing mobile net model
import keras

Loading the model

case 1 : using model with trainable parameters = False;;;
Only training outermost layer

Using sequential API

In [ ]:
mobnet_path = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

In [ ]:
mode2 = hub.KerasLayer(mobnet_path , input_shape = (224,224,3), trainable = False)

In [ ]:
model2_unmodified = tf.keras.Sequential([           
            mode2,
            tf.keras.layers.Dense(units = 2),        # two classes. It can be trained using one class also as it is a binary classification model. The only difference is that the loss function will be different for both types of models     

])

In [ ]:
model2_unmodified.summary()

In [ ]:
model2_unmodified.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), # if no. of classes = 1, then we have used binarycrossentropyloss
    metrics = ['acc']
)

In [ ]:
model2_unmodified.fit(x_t,y_t, epochs = 5)

Epoch 1/5
50/50 [==============================] - 43s 761ms/step - loss: 0.2622 - acc: 0.8725
Epoch 2/5
50/50 [==============================] - 45s 894ms/step - loss: 0.0632 - acc: 0.9762
Epoch 3/5
50/50 [==============================] - 43s 852ms/step - loss: 0.0437 - acc: 0.9831
Epoch 4/5
50/50 [==============================] - 44s 877ms/step - loss: 0.0327 - acc: 0.9894
Epoch 5/5
50/50 [==============================] - 42s 849ms/step - loss: 0.0263 - acc: 0.9937


In [ ]:
score , accuracy = model2_unmodified.evaluate(x_test,y_test)

13/13 [==============================] - 18s 1s/step - loss: 0.0673 - acc: 0.9825


In [ ]:
print("Test loss",score)
print("Test accuracy", accuracy)

Test loss 0.06734900921583176
Test accuracy 0.9825000166893005


Now Using Functional API and training several outer layers for increasing accuracy